# Panel Data

Can run with statsmodels (and dummies) or linearmodels. Statsmodels seems to be a bit more general but also seems to require you to specify the dummies in each regression you run. Linearmodels is not in basic Anaconda.

See following website for helpful examples: https://vincent.codes.finance/posts/panel-ols-standard-errors/

In [1]:
# basic setup
import datetime
from dateutil.relativedelta import relativedelta
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import pytz
import statsmodels.api as sm
import statsmodels.formula.api as smf
import string
import sys

# function to create a tabular table
sys.path.append('submodules/python-tabular-output/')
from tab_general_func import tabularconvert
from tab_general_func import mergetabsecs
from tab_sm_func import getcoefftabmatrix
from tab_sm_func import getparamtabmatrix
from tab_sm_func import getsmresultstable

## Example Without Linearmodels

In [2]:
# get dataset:{{{

ni = 10
nt = 1000

# Example of generating random number distribution
# loc = mean, scale = sd, size = array of distribution
x1 = np.random.normal(loc = 0, scale = 1, size = [ni, nt])
epsilon = np.random.normal(loc = 0, scale = 1, size = [ni, nt])

# basic fixed effect
alphai = np.random.normal(loc = 0, scale = 5, size = [ni])
# time trend by entity
gammai = np.random.normal(loc = 0, scale = 0.5, size = [ni])
gammai = [0] * ni

# time fixed effect
gammat = np.random.normal(loc = 0, scale = 5, size = [nt])

y = np.empty([ni, nt])
idi = np.empty([ni, nt])
timet = np.empty([ni, nt])
for i in range(ni):
    for t in range(nt):
        y[i, t] = alphai[i] + gammat[t] + gammai[i] + x1[i, t] + epsilon[i, t]
        idi[i, t] = i
        timet[i, t] = t
# reshape as variables
y = y.reshape(ni * nt)
x1 = x1.reshape(ni * nt)
idi = idi.reshape(ni * nt)
timet = timet.reshape(ni * nt)

dforiginal = pd.DataFrame({'y': y, 'x1': x1, 'id': idi, 'time': timet})
# get dataset:}}}

df = dforiginal

model0 = smf.ols(formula = 'y ~ x1', data = dforiginal).fit()

model1 = smf.ols(formula = 'y ~ x1 + C(time)', data = dforiginal).fit()

model2 = smf.ols(formula = 'y ~ x1 + C(time) + C(id)', data = dforiginal).fit()

# multiplying and including other necessary terms for multiplication with dummy to make sense
model3 = smf.ols(formula = 'y ~ x1 * C(id)', data = dforiginal).fit()
print(model3.summary())

# multiplying but not including other necessary terms for multiplication with dummy to make sense
model4 = smf.ols(formula = 'y ~ x1 : C(id)', data = dforiginal).fit()
print(model4.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.520
Model:                            OLS   Adj. R-squared:                  0.519
Method:                 Least Squares   F-statistic:                     570.0
Date:                Tue, 28 Jan 2025   Prob (F-statistic):               0.00
Time:                        09:46:18   Log-Likelihood:                -30456.
No. Observations:               10000   AIC:                         6.095e+04
Df Residuals:                    9980   BIC:                         6.110e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -6.5466      0.161    -

## Now Incorporate Linearmodels

In [3]:
import linearmodels as lm

## Alternative Data

In [4]:
# finding impact of annual salary on annual holiday spending

# logentertainment_{i,t} = beta*logsalary_{i,t} + gamma_i + gamma_t + u_{i,t}
# allow logsalary_{i,t} to relate to fixed effect so that there is bias unless fixed effects are controlled for
# for example people who take longer holidays may work less and make less money
# in this case we might have logsalary_{i,t} = -0.1*gamma_i + e_{i,t}

N = 100
T = 100
sd_gammai = 1
sd_gammat = 1
sd_e = 1
sd_u = 1
interactioncoeff = -1
beta = 1

index_i = list(range(N))
index_t = list(range(T))

# set up i dataset and t dataset
gamma_i = np.random.normal(scale=sd_gammai, size=N)
df_i = pd.DataFrame({'gamma_i': gamma_i, 'index_i': index_i})
gamma_t = np.random.normal(scale=sd_gammat, size=T)
df_t = pd.DataFrame({'gamma_t': gamma_t, 'index_t': index_t})

# set up df_it
df = pd.DataFrame(list(product(index_i, index_t)), columns = ['index_i', 'index_t'])
df = df.merge(df_i, on = 'index_i', how = 'left')
df = df.merge(df_t, on = 'index_t', how = 'left')

# add other i,t variables to df
df['e_it'] = np.random.normal(scale=sd_e, size=N*T)
df['u_it'] = np.random.normal(scale=sd_u, size=N*T)
# add interaction with fixed effect
df['logsalary_it'] = df['e_it'] + interactioncoeff * df['gamma_i']
df['logholiday_it'] = beta * df['logsalary_it'] + df['gamma_i'] + df['gamma_t'] + df['u_it']

# set index with i first and t second
dfwithoutindex = df
dfwithindex = df.set_index(['index_i', 'index_t'])

## Linearmodels: OLS

In [5]:
model = lm.PanelOLS.from_formula('logholiday_it ~ logsalary_it', data=dfwithindex).fit()
print(model)

                          PanelOLS Estimation Summary                           
Dep. Variable:          logholiday_it   R-squared:                        0.1736
Estimator:                   PanelOLS   R-squared (Between):             -11.419
No. Observations:               10000   R-squared (Within):               0.2569
Date:                Tue, Jan 28 2025   R-squared (Overall):              0.1736
Time:                        09:46:20   Log-likelihood                -1.867e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      2100.0
Entities:                         100   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                  F(1,9999)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             2100.0
                            

## Linearmodels: OLS with Clustered Fixed Effects

In [6]:
model = lm.PanelOLS.from_formula('logholiday_it ~ logsalary_it + 1', data=dfwithindex).fit(cov_type='clustered', cluster_entity=True)
print(model)
# can also run with statsmodels
# only need the dropna part if I have missing observations
# this is a little different possibly due to the constant
model = smf.ols('logholiday_it ~ logsalary_it', data=dfwithoutindex).fit(cov_type='cluster', cov_kwds={'groups': dfwithoutindex[['logholiday_it', 'logsalary_it', 'index_i']].dropna()['index_i']})
print(model.summary())

                          PanelOLS Estimation Summary                           
Dep. Variable:          logholiday_it   R-squared:                        0.1736
Estimator:                   PanelOLS   R-squared (Between):             -12.620
No. Observations:               10000   R-squared (Within):               0.2568
Date:                Tue, Jan 28 2025   R-squared (Overall):              0.1736
Time:                        09:46:21   Log-likelihood                -1.866e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2099.9
Entities:                         100   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                  F(1,9998)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             197.62
                            

## Linearmodels: Fixed Effects Regression

In [7]:
model = lm.PanelOLS.from_formula('logholiday_it ~ logsalary_it + EntityEffects', data=dfwithindex).fit()
print(model)
# can also run with statsmodels
# only need the dropna part if I have missing observations
model = smf.ols('logholiday_it ~ logsalary_it + C(index_i)', data=dfwithoutindex).fit()
print(model.summary())

                          PanelOLS Estimation Summary                           
Dep. Variable:          logholiday_it   R-squared:                        0.3298
Estimator:                   PanelOLS   R-squared (Between):             -40.687
No. Observations:               10000   R-squared (Within):               0.3298
Date:                Tue, Jan 28 2025   R-squared (Overall):              0.0370
Time:                        09:46:21   Log-likelihood                -1.758e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      4870.1
Entities:                         100   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                  F(1,9899)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             4870.1
                            

## Linearmodels: Clustering on Entity/Fixed Effects

In [9]:
# fixed effects regression clustering on entity effects
model = lm.PanelOLS.from_formula('logholiday_it ~ logsalary_it + EntityEffects', data=dfwithindex).fit(cov_type="clustered", cluster_entity=True)
print(model)
# can also run with statsmodels
# only need the dropna part if I have missing observations
model = smf.ols('logholiday_it ~ logsalary_it + C(index_i)', data=dfwithoutindex).fit(cov_type='cluster', cov_kwds={'groups': dfwithoutindex[['logholiday_it', 'logsalary_it', 'index_i']].dropna()['index_i']})
print(model.summary())

# fixed effects regression clustering on time effects
model = lm.PanelOLS.from_formula('logholiday_it ~ logsalary_it + EntityEffects', data=dfwithindex).fit(cov_type="clustered", cluster_time=True)
print(model)

                          PanelOLS Estimation Summary                           
Dep. Variable:          logholiday_it   R-squared:                        0.3298
Estimator:                   PanelOLS   R-squared (Between):             -40.687
No. Observations:               10000   R-squared (Within):               0.3298
Date:                Tue, Jan 28 2025   R-squared (Overall):              0.0370
Time:                        09:46:21   Log-likelihood                -1.758e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      4870.1
Entities:                         100   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                  F(1,9899)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             5496.7
                            

/shared/venvs/cdc-mdc2/lib/python3.8/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 100, but rank is 1
  warnings.warn('covariance of constraints does not have full '


## Linearmodels: Including Barlett Standard Errors

Similar to Newey-West.

In [10]:
# fixed effects regression clustering on time effects
# can specify bandwidth = 3 to specify bandwith of 3; otherwise its computed automatically
model = lm.PanelOLS.from_formula('logholiday_it ~ logsalary_it + EntityEffects', data=dfwithindex).fit(cov_type="kernel", kernel="bartlett")
print(model)

                          PanelOLS Estimation Summary                           
Dep. Variable:          logholiday_it   R-squared:                        0.3298
Estimator:                   PanelOLS   R-squared (Between):             -40.687
No. Observations:               10000   R-squared (Within):               0.3298
Date:                Tue, Jan 28 2025   R-squared (Overall):              0.0370
Time:                        09:46:21   Log-likelihood                -1.758e+04
Cov. Estimator:        Driscoll-Kraay                                           
                                        F-statistic:                      4870.1
Entities:                         100   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                  F(1,9899)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             8005.4
                            

## Linearmodels: Including Entity and Time Fixed Effects

In [11]:
model = lm.PanelOLS.from_formula('logholiday_it ~ logsalary_it + EntityEffects + TimeEffects', data=dfwithindex).fit()
print(model)

                          PanelOLS Estimation Summary                           
Dep. Variable:          logholiday_it   R-squared:                        0.5017
Estimator:                   PanelOLS   R-squared (Between):             -40.787
No. Observations:               10000   R-squared (Within):               0.3297
Date:                Tue, Jan 28 2025   R-squared (Overall):              0.0363
Time:                        09:46:22   Log-likelihood                -1.396e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      9867.8
Entities:                         100   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                  F(1,9800)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             9867.8
                            

In [12]:
# basic ols
model0 = lm.PanelOLS.from_formula('logholiday_it ~ logsalary_it', data=dfwithindex).fit()

# fixed effects regression
model1 = lm.PanelOLS.from_formula('logholiday_it ~ logsalary_it + EntityEffects', data=dfwithindex).fit()

# fixed effects + time fixed effects regression
model2 = lm.PanelOLS.from_formula('logholiday_it ~ logsalary_it + EntityEffects + TimeEffects', data=dfwithindex).fit()

models = [model0, model1, model2]
tabular = getsmresultstable(models, printtab = True, savename = None, ynames = ['', 'Pooled OLS', 'Fixed', 'Fixed + Time'], coefflist = None, coeffnames = None)

             Pooled OLS Fixed    Fixed + Time
logsalary_it 0.527***   0.995*** 0.996***    
             (0.012)    (0.014)  (0.010)     
N            10000      10000    10000       
$R^2$        0.174      0.330    0.502       
